In [6]:
import re
import spacy
import pandas as pd
from google.colab import drive
from google.colab import files

from os import listdir
from os.path import isfile, join, splitext

import gc

In [2]:
path = '/content/drive/MyDrive/Master_AI/Project2021mai_1/Data/hudoc'
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
def get_file(path):
  with open(path) as file:
    lines = file.readlines()
    lines = [line.rstrip() for line in lines]
  return lines 

def get_text(lines):
  document_text = ""
  i = 0
  for ln in lines:
    document_text = document_text + ln + "\n"
  return document_text

def build_df(lines):
  df = pd.DataFrame(columns = ['Text','Word','POS','Tag','Word_idx','Tag_idx'])
  nlp = spacy.load('en')
  i=0
  for ln in lines:
    tokens = nlp(ln)
    tokens1 = [token.text for token in tokens]
    tags = ['O']*len(tokens1)
    tags_id = [0]*len(tokens1)
    df.loc[i] = pd.Series({'Sentence': i,'Text':ln, 'Word':tokens1, 'Tag':tags, 'Tag_idx':tags_id})
    i=i+1
  return df

In [ ]:
with open('/content/drive/MyDrive/Master_AI/Project2021mai_1/Data/hudoc/Text/file001.txt') as file:
    lines = file.readlines()
    lines = [line.rstrip() for line in lines]

document_text = ""
i = 0
for ln in lines:
    document_text = document_text + ln + "\n"
    #print(ln)

In [ ]:
df = pd.DataFrame(columns = ['Text','Word','POS','Tag','Word_idx','Tag_idx'])
nlp = spacy.load('en')
i=0
for ln in lines:
    tokens = nlp(ln)
    tokens1 = [token.text for token in tokens]
    tags = ['O']*len(tokens1)
    tags_id = [0]*len(tokens1)
    df.loc[i] = pd.Series({'Sentence': i,'Text':ln, 'Word':tokens1, 'Tag':tags, 'Tag_idx':tags_id})
    i=i+1

In [10]:
LABLE_MODE = 'BIO'

base_labels = ['LAW','CARDINAL','DATE','GPE','ORG',
               'WORK_OF_ART','PERSON','NORP','LOC','COURT',
               'CASE','JUDGE','REGISTRAR','APPLICATION','ARTICLE',
               'SECTION','PARAGRAPH','PROTOCOL',
               'VICTIMS','INVESTIGATORS','STATEMENTS','SECRETARY','LAWYER','DEFENDANT','PROSECUTOR','PETITION']

def create_lables(base_labels,type = 'BILOU'):
  label_list = ['O'] * (len(base_labels)+1)
  label_ids = [0] * (len(base_labels)+1)
  label_modificator = ['']
  if (type == 'BILOU'):
    label_modificator = ['B-','I-','L-','U-']
    label_list = ['O'] * (len(base_labels)*4+1)
    label_ids = [0] * (len(base_labels)*4+1)
  elif (type == 'BIO'):
    label_modificator = ['B-','I-']
    label_list = ['O'] * (len(base_labels)*2+1)
    label_ids = [0] * (len(base_labels)*2+1)

  ii = 1
  for i in base_labels:
    jj = 0
    for j in label_modificator:
      label_list[ii+jj] = j+i
      label_ids[ii+jj] = jj+ii
      jj = jj + 1
    ii = ii + len(label_modificator)

  return label_list, label_ids

def get_ne_label(base=1,mod=0,type = 'BILOU'): # only for entrys higher than 0
  if (type == 'BILOU'):
      base = ((base-1)*4) + 1
  elif (type == 'BIO'):
    base = ((base-1)*2) + 1
  return base + mod

label_list, label_ids = create_lables(base_labels,type = LABLE_MODE)
print(label_list)
print(label_ids)
#print(get_ne_label(base=3,mod=2,type = 'BILOU'))

['O', 'B-LAW', 'I-LAW', 'B-CARDINAL', 'I-CARDINAL', 'B-DATE', 'I-DATE', 'B-GPE', 'I-GPE', 'B-ORG', 'I-ORG', 'B-WORK_OF_ART', 'I-WORK_OF_ART', 'B-PERSON', 'I-PERSON', 'B-NORP', 'I-NORP', 'B-LOC', 'I-LOC', 'B-COURT', 'I-COURT', 'B-CASE', 'I-CASE', 'B-JUDGE', 'I-JUDGE', 'B-REGISTRAR', 'I-REGISTRAR', 'B-APPLICATION', 'I-APPLICATION', 'B-ARTICLE', 'I-ARTICLE', 'B-SECTION', 'I-SECTION', 'B-PARAGRAPH', 'I-PARAGRAPH', 'B-PROTOCOL', 'I-PROTOCOL', 'B-VICTIMS', 'I-VICTIMS', 'B-INVESTIGATORS', 'I-INVESTIGATORS', 'B-STATEMENTS', 'I-STATEMENTS', 'B-SECRETARY', 'I-SECRETARY', 'B-LAWYER', 'I-LAWYER', 'B-DEFENDANT', 'I-DEFENDANT', 'B-PROSECUTOR', 'I-PROSECUTOR', 'B-PETITION', 'I-PETITION']
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52]


In [32]:
re_dict = []

date_re = [r'\d{2} \w+ \d{4}\b',r'[A-Z]\w+ \d{4}\b']
court_re =[r'\w+ SECTION\b', r'European Court of Human Rights\b'] 
judge_re =[]#['Josep Casadevall', ' Alvina Gyulumyan', 'J n ikuta', 'Luis L pez Guerra', 'Johannes Silvis', 'Valeriu Gri co', 'Iulia Antoanella Motoc']
registrar_re=[]
case_re = [] #To unstabel  
cases_re = [r'\w+ v. \w+',r'Cases',r'Case']
application_re = [r'no. \d{4,}/\d{2}\b',r'\d{4,}/\d{2}\b',r'application']
petition_re = [r'petition',r'request',r'requests']
article_re = [r'Article \d+\b']
peragraph_re = [r'§§ \d+',r'§ \d+\b']

def get_board_re(lines):
  person_str = ""
  word = "judges"
  #Get line with judges
  person_str = ""
  word = "judges"
  for ln in lines:
    if word in ln: 
      #print('success')
      person_str += ln
      #print(person_str)
      break
  #Clean line
  person_str = person_str.replace('and ', '')
  person_str = person_str.replace('President, ', '')
  #split judges
  judg_l = []
  regi_l = []
  try:
    person_list = person_str.split(", judges, ")
    judg_l = person_list[0].split(", ")
    regi_l = person_list[1].split(", ")
    regi_l = regi_l[:-1]
    return judg_l, regi_l
  except:
    return judg_l, regi_l

def get_defined_NE(target_string,tags_re,tag_id,tag_label):
  re_str = "("
  for word in tags_re:
    re_str += word+"|"
  re_str = re_str[:-1]
  re_str += ")"
  #print(re_str)
  comp_re = re.compile(re_str)
  for match in re.finditer(comp_re, target_string):
    dict_entry = [match.start(), match.group(),tag_id,tag_label] #
    re_dict.append(dict_entry)
    #print(dict_entry)
  return

def build_NE_Lst(target_string,lines):
  #Preprocessing
  judge_re = []
  registrar_re = []
  judge_re, registrar_re = get_board_re(lines)
  #Find NE
  get_defined_NE(target_string,date_re,         3,'DATE')
  get_defined_NE(target_string,court_re,        10,'COURT')
  if(len(judge_re)>0):
    get_defined_NE(target_string,judge_re,        12,'JUDGE')
  if(len(registrar_re)>0):
    get_defined_NE(target_string,registrar_re,    13,'REGISTRAR')
  get_defined_NE(target_string,cases_re,        11,'CASE')
  get_defined_NE(target_string,application_re,  14,'APPLICATION')
  get_defined_NE(target_string,petition_re,     26,'PETITION')
  get_defined_NE(target_string,article_re,      15,'ARTICLE')
  get_defined_NE(target_string,peragraph_re,    17,'PARAGRAPH')
  #Sort Lable List
  re_dict.sort(key=lambda x: x[0])

  return

#build_NE_Lst(document_text)

### old

In [ ]:
### OLD ###

re_dict = []

def get_Court(target_string):
  print('get_Court')
  application_rules = [r'\w+ SECTION\b', r'European Court of Human Rights\b']
  print("(%s|%s)" % (application_rules[0], application_rules[1]))
  application_re = re.compile("(%s|%s)" % (application_rules[0], application_rules[1]))
  for match in re.finditer(application_re, target_string):
    dict_entry = [match.start(), match.group(),10,'COURT']
    re_dict.append(dict_entry)
    #print(dict_entry)
  return

#get_Court(document_text)

def get_Case(target_string):
  return

def get_Cases(target_string):
  print('get_Cases')
  application_rules = [r'\w+ v. \w+',r'Cases',r'Case']
  application_re = re.compile("(%s|%s|%s)" % (application_rules[0], application_rules[1], application_rules[2]))
  for match in re.finditer(application_re, target_string):
    dict_entry = [match.start(), match.group(),11,'CASE']
    re_dict.append(dict_entry)
    #print(dict_entry)
  return

def get_Application(target_string):
  print('get_Application')
  application_rules = [r'no. \d{4,}/\d{2}\b',r'\d{4,}/\d{2}\b',r'application']
  application_re = re.compile("(%s|%s|%s)" % (application_rules[0], application_rules[1], application_rules[2]))
  for match in re.finditer(application_re, target_string):
    dict_entry = [match.start(), match.group(),14,'APPLICATION']
    re_dict.append(dict_entry)
    #print(dict_entry)
  return 

def get_Petition(target_string):
  print('get_Petition')
  application_rules = [r'petition',r'request',r'requests']
  application_re = re.compile("(%s|%s|%s)" % (application_rules[0], application_rules[1], application_rules[2]))
  for match in re.finditer(application_re, target_string):
    dict_entry = [match.start(), match.group(),14,'PETITION']
    re_dict.append(dict_entry)
    #print(dict_entry)
  return 

def get_Judge(target_string):
  return

def get_Date(target_string):
  print('get_Date')
  date_rules = [r'\d{2} \w+ \d{4}\b',r'[A-Z]\w+ \d{4}\b']
  date_re = re.compile(r"(\d{1,} \w+ \d{4}\b|[A-Z]\w+ \d{4}\b)")#("(%s|%s)" % (date_rules[0], date_rules[1]))
  for match in re.finditer(date_re, target_string):
    dict_entry = [match.start(), match.group(),3,'DATE']
    re_dict.append(dict_entry)
    #print(dict_entry)
  return

def get_Article(target_string):
  print('get_Article')
  article_rules = [r'Article \d+\b']
  article_re = re.compile("(%s)" % (article_rules[0]))
  for match in re.finditer(article_re, target_string):
    dict_entry = [match.start(), match.group(),15,'ARTICLE']
    re_dict.append(dict_entry)
    #print(dict_entry)
  return

def get_Paragraph(target_string):
  print('get_Paragraph')
  re1 = r'§§ \d+'
  re2 = r'§ \d+\b'
  paragraph_re = re.compile("(%s|%s)" % (re1, re2))
  for match in re.finditer(paragraph_re, target_string):
    dict_entry = [match.start(), match.group(),17,'PARAGRAPH']
    re_dict.append(dict_entry)
    #print(dict_entry)
  return


def build_NE__List(document_text):
  get_Court(document_text)
  get_Petition(document_text)
  get_Cases(document_text)
  get_Application(document_text)
  get_Article(document_text)
  get_Paragraph(document_text)
  get_Date(document_text)

  re_dict.sort(key=lambda x: x[0])
  
  return

#build_NE__List(document_text)



### Neuer Abschnitt

In [ ]:
print('\n'.join([' - '.join(['{:4}'.format(item) for item in row]) 
      for row in re_dict]))

   0 - THIRD SECTION -   10 - COURT
  32 - BUCIA v. ROMANIA -   11 - CASE
  62 - no. 32185/04 -   14 - APPLICATION
 115 - 16 December 2014 -    3 - DATE
 181 - July 2014 -    3 - DATE
 245 - Article 44 -   15 - ARTICLE
 256 - § 2  -   17 - PARAGRAPH
 345 - Bucia v. Romania -   11 - CASE
 367 - European Court of Human Rights -   10 - COURT
 450 - Josep Casadevall -   12 - JUDGE
 479 - Alvina Gyulumyan -   12 - JUDGE
 497 - J n ikuta -   12 - JUDGE
 508 - Luis L pez Guerra -   12 - JUDGE
 527 - Johannes Silvis -   12 - JUDGE
 544 - Valeriu Gri co -   12 - JUDGE
 560 - Iulia Antoanella Motoc -   12 - JUDGE
 596 - Santiago Quesada -   13 - REGISTRAR
 666 - 10 June 2014 -    3 - DATE
 781 - application -   14 - APPLICATION
 794 - no. 32185/04 -   14 - APPLICATION
 852 - Article 34 -   15 - ARTICLE
1047 - 15 July 2004 -    3 - DATE
1146 - December 2005 -    3 - DATE
1252 - application -   14 - APPLICATION
1557 - Palumbo v. Italy -   11 - CASE
1575 - 27 February 1992 -    3 - DATE
1593 - § 2 

In [ ]:
print(document_text)

THIRD SECTION
CASE OF BUCEA AND BUCIA v. ROMANIA
(Application no. 32185/04)
JUDGMENT
This version was rectified on 16 December 2014 under Rule 81 of the Rules of Court
STRASBOURG
1 July 2014
FINAL
01/10/2014
This judgment has become final under Article 44 2 of the Convention. It may be subject to editorial revision.
In the case of Bucea and Bucia v. Romania,
The European Court of Human Rights (Third Section), sitting as a Chamber compomsed of:
Josep Casadevall, President, Alvina Gyulumyan, J n ikuta, Luis L pez Guerra, Johannes Silvis, Valeriu Gri co, Iulia Antoanella Motoc, judges, and Santiago Quesada, Section Registrar,
Having deliberated in private on 10 June 2014,
Delivers the following judgment, which was adopted on that date:
PROCEDURE
The case originated in an application (no. 32185/04) against Romania lodged with the Court under Article 34 of the Convention for the Protection of Human Rights and Fundamental Freedoms ( the Convention ) by two Romanian nationals, Mr Ioan Bucea a

In [34]:
dict_phrase_idx = 0
dict_word_idx = 0
line_idx = 0
word_idx = 0

def reset_filling():
  dict_phrase_idx = 0
  dict_word_idx = 0
  line_idx = 0
  word_idx = 0

def enhance_tag(tag_id,phrase_size=1,mode='BILOU'):# LABLE_MODE
  f_lable = get_ne_label(base=tag_id,type=mode)
  l_lable = f_lable + 1

  if (mode == 'BILOU'):
    l_lable = f_lable + 4
  elif (mode == 'BIO'):
    l_lable = f_lable + 2

  lid = label_ids[f_lable:l_lable]
  lna = label_list[f_lable:l_lable]

  base_lable_id = lid[0]
  base_lable_na = lna[0]

  if (mode == 'BILOU'):
    base_lable_id = lid[1]
    base_lable_na = lna[1]
  elif (mode == 'BIO'):
    base_lable_id = lid[1]
    base_lable_na = lna[1]

  tag_ids = [base_lable_id]*phrase_size
  tag_labels = [base_lable_na]*phrase_size

  if(mode == 'BILOU' and phrase_size <= 1):
    tag_ids[0] = lid[3]
    tag_labels[0] = lna[3]
  elif(mode == 'BILOU' or mode == 'BIO'):
    tag_ids[0] = lid[0]
    tag_labels[0] = lna[0]

  if (mode == 'BILOU'):
    tag_ids[-1] = lid[2]
    tag_labels[-1] = lna[2]
  return tag_ids, tag_labels

enhance_tag(1,5)

def get_next_tag(keylist, idx):
  nlp = spacy.load('en')
  phrase = keylist[idx][1] 
  tag_id = keylist[idx][2]
  tag_name = keylist[idx][3]
  tokens = nlp(phrase)
  words = [token.text for token in tokens]
  tag_ids,tag_labels = enhance_tag(tag_id,len(words),mode=LABLE_MODE)
  return words, tag_ids, tag_labels


def label_line(df_idx, search_phrase, tag_ids, tag_labels):
  kp_start = 0
  dict_word_idx = 0
  row = df.iloc[df_idx]
  words = row['Word']
  for i in range(len(words)):
    if(search_phrase[dict_word_idx] == words[i]):
      dict_word_idx +=1
    else:
      dict_word_idx = 0
      kp_start = (i+1)
    if(dict_word_idx >= len(search_phrase)):
      line_tags = row['Tag']
      line_tags_ids = row['Tag_idx']
      for k in range(len(search_phrase)):
        line_tags[k+kp_start] = tag_labels[k]
        line_tags_ids[k+kp_start] = tag_ids[k]
      df.loc[df.index[df_idx], 'Tag'] = line_tags
      df.loc[df.index[df_idx], 'Tag_idx'] = line_tags_ids
      return True
  return False

def lable_text():
  dict_id = 0
  akt_phrase,akt_tagId,akt_tagName = get_next_tag(re_dict, dict_id)
  i = 0
  print(' -> ',end=' ')
  print(i,end=' ')
  while (i < len(df)):
    #print(i)
    if(label_line(i, akt_phrase, akt_tagId, akt_tagName)):
      dict_id +=1
      if(dict_id >= len(re_dict)):
        break
      akt_phrase,akt_tagId,akt_tagName = get_next_tag(re_dict, dict_id)
    else:
      i+=1
      if((i%10)==0):
        print(i,end=' ')
  return

#lable_text()



In [17]:
path = '/content/drive/MyDrive/Group 18 - MRP1/Dataset/HUDOC/Text'
onlyfiles = [f for f in listdir(path) if isfile(join(path, f))]
full_path = path + '/' + onlyfiles[0]
print(full_path)

/content/drive/MyDrive/Group 18 - MRP1/Dataset/HUDOC/Text/001-94452.txt


In [18]:
lines = get_file(full_path)
document_text = get_text(lines)

In [ ]:
print(document_text)

In [ ]:
path = '/content/drive/MyDrive/Group 18 - MRP1/Dataset/HUDOC/Text'
onlyfiles = [f for f in listdir(path) if isfile(join(path, f))]

#print(onlyfiles)

for file_name in onlyfiles:
  try:
    full_path = path + '/' + file_name
    filename, file_extension = splitext(file_name)

    print(filename,end=': ')

    lines = get_file(full_path)
    document_text = get_text(lines)
    df = build_df(lines)

    re_dict = []
    build_NE_Lst(document_text,lines)
    print('entities',end=': ')
    print(len(re_dict),end=' ')

    reset_filling()
    lable_text()

    save_path = '/content/drive/MyDrive/Group 18 - MRP1/Dataset/HUDOC/Annotated/'+filename+'.csv'
    df.to_csv(save_path)
    print(' SAVE')
  except:
    #delete file
    print('fail')
    continue

001-94452: entities: 312  ->  0 10 20 30 40 50 60 70 80 90 100 110 120 130 140 150 160 170 180 190  SAVE
001-94301: entities: 325  ->  0 10 20 30 40 50 60 70 80 90 100 110 120 130 140 150 160 170 180 190 200 210 220 230 240 250  SAVE
001-94585: entities: 338  ->  0 10 20 30 40 50 60 70 80 90 100 110 120 130 140 150 160 170 180 190 200 210 220 230 240 250  SAVE
001-94533: entities: 241  ->  0 10 20 30 40 50 60 70 80 90 100 110 120 130 140  SAVE
001-94218: entities: 152  ->  0 10 20 30 40 50 60 70 80 90 100 110 120  SAVE
001-94636: entities: 103  ->  0 10 20 30 40 50 60 70 80  SAVE
001-94831: entities: 68  ->  0 10 20 30 40  SAVE
001-94222: entities: 159  ->  0 10 20 30 40 50 60 70 80 90 100 110 120 130 140 150 160 170  SAVE
001-94733: entities: 61  ->  0 10 20 30  SAVE
001-94446: entities: 109  ->  0 10 20 30 40 50 60 70  SAVE
001-94456: entities: 162  ->  0 10 20 30 40 50 60 70 80 90 100  SAVE
001-94634: entities: 96  ->  0 10 20 30 40 50 60 70 80 90  SAVE
001-94752: entities: 166  -> 

In [ ]:
df.head(30)

,Text,Word,POS,Tag,Word_idx,Tag_idx
0,THIRD SECTION,"[THIRD, SECTION]",NaN,"[B-COURT, L-COURT]",NaN,"[37, 39]"
1,CASE OF BUCEA AND BUCIA v. ROMANIA,"[CASE, OF, BUCEA, AND, BUCIA, v., ROMANIA]",NaN,"[O, O, O, O, B-CASE, I-CASE, L-CASE]",NaN,"[0, 0, 0, 0, 41, 42, 43]"
2,(Application no. 32185/04),"[(, Application, no, ., 32185/04, )]",NaN,"[O, O, B-APPLICATION, I-APPLICATION, L-APPLICA...",NaN,"[0, 0, 53, 54, 55, 0]"
3,JUDGMENT,[JUDGMENT],NaN,[O],NaN,[0]
4,This version was rectified on 16 December 2014...,"[This, version, was, rectified, on, 16, Decemb...",NaN,"[O, O, O, O, O, B-DATE, I-DATE, L-DATE, O, O, ...",NaN,"[0, 0, 0, 0, 0, 9, 10, 11, 0, 0, 0, 0, 0, 0, 0..."
5,STRASBOURG,[STRASBOURG],NaN,[O],NaN,[0]
6,1 July 2014,"[1, July, 2014]",NaN,"[O, B-DATE, L-DATE]",NaN,"[0, 9, 11]"
7,FINAL,[FINAL],NaN,[O],NaN,[0]
8,01/10/2014,[01/10/2014],NaN,[O],NaN,[0]
9,This judgment has become final under Article 4...,"[This, judgment, has, become, final, under, Ar...",NaN,"[O, O, O, O, O, O, B-ARTICLE, L-ARTICLE, B-PAR...",NaN,"[0, 0, 0, 0, 0, 0, 57, 59, 65, 67, 0, 0, 0, 0,..."


In [ ]:
save_path = '/content/drive/MyDrive/Master_AI/Project2021mai_1/Data/hudoc/annotated/faile001.csv'
document_df.to_csv(save_path, sep='\t')

In [ ]:
for i in range(len(df)):
  row = df.iloc[i]
  print(row['Word'])


['THIRD', 'SECTION']
['CASE', 'OF', 'BUCEA', 'AND', 'BUCIA', 'v.', 'ROMANIA']
['(', 'Application', 'no', '.', '32185/04', ')']
['JUDGMENT']
['This', 'version', 'was', 'rectified', 'on', '16', 'December', '2014', 'under', 'Rule', '81', 'of', 'the', 'Rules', 'of', 'Court']
['STRASBOURG']
['1', 'July', '2014']
['FINAL']
['01/10/2014']
['This', 'judgment', 'has', 'become', 'final', 'under', 'Article', '44', '§', '2', 'of', 'the', 'Convention', '.', 'It', 'may', 'be', 'subject', 'to', 'editorial', 'revision', '.']
['In', 'the', 'case', 'of', 'Bucea', 'and', 'Bucia', 'v.', 'Romania', ',']
['The', 'European', 'Court', 'of', 'Human', 'Rights', '(', 'Third', 'Section', ')', ',', 'sitting', 'as', 'a', 'Chamber', 'compomsed', 'of', ':']
['Josep', 'Casadevall', ',', 'President', ',', 'Alvina', 'Gyulumyan', ',', 'J', 'n', 'ikuta', ',', 'Luis', 'L', 'pez', 'Guerra', ',', 'Johannes', 'Silvis', ',', 'Valeriu', 'Gri', 'co', ',', 'Iulia', 'Antoanella', 'Motoc', ',', 'judges', ',', 'and', 'Santiago', 'Qu